In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MTx_Net(nn.Module):
    def __init__(self, classes):
        super(MTx_Net, self).__init__()
        self.rescale = nn.Sequential(
            nn.Conv2d(3, 128, 1, 1, 0),
            nn.BatchNorm2d(128),
        )

        self.block0 = nn.Sequential(
            nn.Conv2d(128, 32, 3, 1, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 16, 3, 2, 1),
            nn.BatchNorm2d(16),
        )

        self.block1 = self._create_block(16, 96, 24)
        self.block2 = self._create_block(24, 144, 36)
        self.block3 = self._create_block(36, 216, 54)
        self.block4 = self._create_block(54, 324, 81)

        self.final = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(81, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Dropout(0.4),
            nn.Flatten(),
            nn.Linear(256, classes),
        )

    def _create_block(self, in_channels, exp_channels, out_channels):
        return nn.Sequential(
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, exp_channels, 1, 1, 0),
            nn.BatchNorm2d(exp_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(exp_channels, exp_channels, 3, 2, 1, groups=exp_channels),
            nn.BatchNorm2d(exp_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(exp_channels, out_channels, 1, 1, 0),
            nn.BatchNorm2d(out_channels),
            nn.Conv2d(out_channels, out_channels, 1, 2, 0),
            nn.BatchNorm2d(out_channels),
        )
    def forward(self, x):
        x = self.rescale(x)
        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.final(x)
        return x


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
# from MTx_Net import MTx_Net

SIZE = 128
batch_size = 64

# image_size = (SIZE, SIZE)
# input_size = (SIZE, SIZE, 3)

device = "cuda" if torch.cuda.is_available() else "cpu"

initialization_path = r"C:\Users\14282\OneDrive\桌面\MeatTexture\ILSVRC2012_120class"
pre_train_path = r"C:\Users\14282\OneDrive\桌面\MeatTexture\Pre_Train"
lamb_train_path = r"C:\Users\14282\OneDrive\桌面\MeatTexture\Lamb\train"
lamb_val_path = r"C:\Users\14282\OneDrive\桌面\MeatTexture\Lamb\val"
beef_train_path = r"C:\Users\14282\OneDrive\桌面\MeatTexture\Beef\train"
beef_val_path = r"C:\Users\14282\OneDrive\桌面\MeatTexture\Beef\val"
model_save_preTrain = "preTrain.pth"
model_save_Lamb = "lamb.pth"
model_save_Beef = "beef.pth"

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(SIZE, scale=(0.85, 1.2)),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize(SIZE),
    transforms.CenterCrop(SIZE),
    transforms.ToTensor()
])

# Build model
model_pre = MTx_Net(120).to(device)  

# Training -- pre-train 1
optimizer = optim.Adam(model_pre.parameters())
criterion = nn.CrossEntropyLoss().to(device)

def scheduler(optimizer, epoch, lr_decay_epoch=50):
    if epoch % lr_decay_epoch == 0 and epoch != 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] /= 2
            print("lr changed to {}".format(param_group['lr']))
    return optimizer

initialization_data = datasets.ImageFolder(initialization_path, transform=train_transform)
initialization_loader = DataLoader(initialization_data, batch_size=batch_size, shuffle=True)


for epoch in range(2):
    print("Epoch:", epoch)
    optimizer = scheduler(optimizer, epoch)
    for batch_idx, (data, target) in enumerate(initialization_loader):
        # import pdb; pdb.set_trace()
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model_pre(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
# Save model
torch.save(model_pre.state_dict(), 'initialization_MTx128.pth')

Epoch: 0
Epoch: 1


In [8]:
# Convert to a 3-class model
model = MTx_Net(3)

model.load_state_dict(model_pre.state_dict(), strict=False)

# Training -- pre-train
optimizer = optim.Adam(model.parameters(), lr=0.0001)

pre_train_data = datasets.ImageFolder(pre_train_path, transform=train_transform)
pre_train_loader = DataLoader(pre_train_data, batch_size=batch_size, shuffle=True)

for epoch in range(20):
    for batch_idx, (data, target) in enumerate(pre_train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

torch.save(model.state_dict(), model_save_preTrain)

RuntimeError: Error(s) in loading state_dict for MTx_Net:
	size mismatch for final.7.weight: copying a param with shape torch.Size([120, 256]) from checkpoint, the shape in current model is torch.Size([3, 256]).
	size mismatch for final.7.bias: copying a param with shape torch.Size([120]) from checkpoint, the shape in current model is torch.Size([3]).

In [ ]:
# Training -- lamb
optimizer = optim.Adam(model.parameters(), lr=0.0002)

lamb_train_data = datasets.ImageFolder(lamb_train_path, transform=train_transform)
lamb_train_loader = DataLoader(lamb_train_data, batch_size=batch_size, shuffle=True)

lamb_val_data = datasets.ImageFolder(lamb_val_path, transform=val_transform)
lamb_val_loader = DataLoader(lamb_val_data, batch_size=batch_size, shuffle=False)

for epoch in range(20):
    optimizer = scheduler(optimizer, epoch, lr_decay_epoch=2)
    for batch_idx, (data, target) in enumerate(lamb_train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

torch.save(model.state_dict(), model_save_Lamb)

In [ ]:
# Training -- beef
optimizer = optim.Adam(model.parameters(), lr=0.0001)

beef_train_data = datasets.ImageFolder(beef_train_path, transform=train_transform)
beef_train_loader = DataLoader(beef_train_data, batch_size=batch_size, shuffle=True)

beef_val_data = datasets.ImageFolder(beef_val_path, transform=val_transform)
beef_val_loader = DataLoader(beef_val_data, batch_size=batch_size, shuffle=False)

for epoch in range(10):
    optimizer = scheduler(optimizer, epoch, lr_decay_epoch=1)
    for batch_idx, (data, target) in enumerate(beef_train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

torch.save(model.state_dict(), model_save_Beef)

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import numpy as np

model_path = './models/beef.pth'
image_path = './Beef/val/0/1a.jpg'
model = torch.load(model_path, map_location=torch.device('cpu'))
model.eval()
print('Model loaded.')

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img = Image.open(image_path)
img = transform(img)
img = img.unsqueeze(0)

with torch.no_grad():
    preds = F.softmax(model(img), dim=1)

result_label = np.argmax(preds.numpy())
if result_label == 0:
    print('Genuine')
elif result_label == 1:
    print('Fake')
else:
    print('Irrelevant')